In [ ]:
# OpenAI API to communicate with the LLM model:
from langchain_openai import ChatOpenAI
# to create a langchain agent:
from langchain.agents import create_agent
# a decorator to create tools, that can be simply functions that the LLM can use:
from langchain.tools import tool
# to add memory to the chats, so the LLM can remember previous messages:
from langgraph.checkpoint.memory import InMemorySaver
# to define classes for return data and session storage:
from dataclasses import dataclass

SYSTEM_PROMPT = """You are a microcontroller (MCU) expert in designing an RC filter by choosing final values for: "fitler resistance R_filter" and "filter capacitance C_filter"
You have access to these tools: get_mcu_info, get_R_filter, get_cutoff_frequency, get_new_C

1) Initially, make sure you get the following four parameters from the user: MCU chip name, sampling frequency, maximum voltage drop and the desired cutoff frequency. If any of these 4 are missing, ask the user to provide in the next message.
2) Get the mcu parameters. Then determine the filter resistance. Now you can get the cutoff frequency, assuming initial filter capacitance of C_filter = 100e-9.
3) If the cutoff frequency you get is more than 5% different from the frequency asked by the user, use the get_new_C tool to get a new value for C and call the get_cutoff_frequency again.
Repeat (3) until you reach closer than 5% from the desired cutoff frequency"""

@tool
def get_mcu_info(chip_name: str) -> dict:
    """returns important MCU parameters: i_mcu: ADC input current of the MCU, c_mcu: ADC sampling capacitor of the MCU"""
    if chip_name == "TI_C2000":
        i = 1e-6
        c = 1e-6
    else:
        i = 10e-6
        c = 20e-6
    return {"i_mcu" : i, "c_mcu": c}

@tool
def get_R_filter(i_mcu: float, c_mcu: float, sampling_frequency: float, max_voltage_drop: float) -> float:
    """Calculate the filter resistance"""
    Rmax = 1 / 4 / sampling_frequency / c_mcu
    Rmax = min(Rmax, max_voltage_drop / i_mcu)
    Rmin = 1000
    R_filter = (Rmax + Rmin) / 2
    return round(R_filter,2)

@tool
def get_cutoff_frequency(R: float, C: float) -> float:
    """Calculates the cutoff frequency based on R and C input values"""
    return round(1 / 2 / 3.14 / R / C,9)

@tool
def get_new_C(C: float, cutoff_frequency: float, desired_frequency: float) -> float:
    """Compare cutoff frequency with desired cutoff frequency and give a new value for C to help reach the desired cutoff frequency value"""
    new_C = C * (1 + (cutoff_frequency / desired_frequency - 1) / 2)
    return round(new_C,9)

model = ChatOpenAI(
  api_key="API KEY",
  base_url="https://openrouter.ai/api/v1",
  model="openrouter/polaris-alpha",
)

# a schema where the model can return the exact parameters, in addition to the text response
@dataclass
class ResponseFormat:
    """Response schema for the agent."""
    punny_response: str
    final_cutoff_frequency: str | None = None
    R_filter: str | None = None
    final_C_filter: str | None = None

# a context to store 
@dataclass
class Context:
    """Custom runtime context schema."""
    user_id: str

agent = create_agent(
    model=model,
    system_prompt=SYSTEM_PROMPT,
    tools=[get_mcu_info, get_R_filter, get_cutoff_frequency, get_new_C],
    context_schema=Context,
    response_format=ResponseFormat,
    checkpointer=InMemorySaver()
)

query = "I want to design an RC filter to use with MCU chip named 'TI_C2000'. I want the cutoff frequency to be at 4000 Hz and maximum voltage drop of 0.1 Volts. The sampling rate is 10000 Hz."

response = agent.invoke(
    {"messages": [{"role": "user", "content": query}]},
    config={"configurable": {"thread_id": "1"}},
    context=Context(user_id="1")
)

print(response['structured_response'])

ResponseFormat(punny_response='Here are your tuned RC values—no resistance to good filtering here!', final_cutoff_frequency='3883.8 Hz', R_filter='512.5 Ohms', final_C_filter='80 nF')


In [5]:
response

{'messages': [HumanMessage(content="I want to design an RC filter to use with MCU chip named 'TI_C2000'. I want the cutoff frequency to be at 4000 Hz and maximum voltage drop of 0.1 Volts. The sampling rate is 10000 Hz.", additional_kwargs={}, response_metadata={}, id='f14d9db6-3a05-406b-81ec-00247f8bddff'),
  AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 29, 'prompt_tokens': 596, 'total_tokens': 625, 'completion_tokens_details': {'accepted_prediction_tokens': None, 'audio_tokens': None, 'reasoning_tokens': 0, 'rejected_prediction_tokens': None}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'openrouter/polaris-alpha', 'system_fingerprint': None, 'id': 'gen-1762895595-6Ke61OK1on94qhupdxJM', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--805a242a-6d38-4a50-a68c-f90d503389a9-0', tool_calls=[{'name': 'get_mcu_info', 'args': {'chip_name': 'TI

In [6]:
from langchain_core.messages import AIMessage

n_iterations = 0
n_steps = 0
for i,v in enumerate(response["messages"]):
    if  isinstance(v, AIMessage):
        n_steps += 1
        print(f"{n_steps}) {v.tool_calls[0]["name"]}({v.tool_calls[0]["args"]}) = {response["messages"][i+1].content}\n")

        if v.tool_calls[0]["name"] == "get_new_C": 
            n_iterations +=1

print(f"The model ran {n_iterations} iterations to reach the final result!")

1) get_mcu_info({'chip_name': 'TI_C2000'}) = {"i_mcu": 1e-06, "c_mcu": 1e-06}

2) get_R_filter({'i_mcu': 1e-06, 'c_mcu': 1e-06, 'sampling_frequency': 10000, 'max_voltage_drop': 0.1}) = 512.5

3) get_cutoff_frequency({'R': 512.5, 'C': 1e-07}) = 3107.037439801

4) get_new_C({'C': 1e-07, 'cutoff_frequency': 3107.037439801, 'desired_frequency': 4000}) = 8.9e-08

5) get_cutoff_frequency({'R': 512.5, 'C': 8.9e-08}) = 3491.053303147

6) get_new_C({'C': 8.9e-08, 'cutoff_frequency': 3491.053303147, 'desired_frequency': 4000}) = 8.3e-08

7) get_cutoff_frequency({'R': 512.5, 'C': 8.3e-08}) = 3743.41860217

8) get_new_C({'C': 8.3e-08, 'cutoff_frequency': 3743.41860217, 'desired_frequency': 4000}) = 8e-08

9) get_cutoff_frequency({'R': 512.5, 'C': 8e-08}) = 3883.796799751

10) ResponseFormat({'punny_response': 'Here are your tuned RC values—no resistance to good filtering here!', 'final_cutoff_frequency': '3883.8 Hz', 'R_filter': '512.5 Ohms', 'final_C_filter': '80 nF'}) = Returning structured resp

In [10]:
query = "I want to design an RC filter to use with MCU chip named 'TI_C2000'. I want the cutoff frequency to be at 20 kHz and maximum voltage drop of 0.1 Volts."

response = agent.invoke(
    {"messages": [{"role": "user", "content": query}]},
    config={"configurable": {"thread_id": "21"}},
    context=Context(user_id="1")
)
print(response['structured_response'])

ResponseFormat(punny_response='Sure thing—let’s dial this filter in just right. First I need one missing piece: what is your ADC sampling frequency (in Hz) for the TI_C2000 in this design? Once I have that, I’ll calculate R_filter and C_filter to hit ~20 kHz cutoff within 5%.', final_cutoff_frequency=None, R_filter=None, final_C_filter=None)


In [11]:
query = "The sampling frequency is 50 kHz"

response = agent.invoke(
    {"messages": [{"role": "user", "content": query}]},
    config={"configurable": {"thread_id": "21"}},
    context=Context(user_id="1")
)
print(response['structured_response'])

ResponseFormat(punny_response='We’ve ohm-ost nailed it! Here are your RC values that hit the target cutoff nicely.', final_cutoff_frequency='19805 Hz', R_filter='502.5 Ohms', final_C_filter='16 nF')
